# YOLOv4 on Berley DeepDrive Dataset

The goal of this colab notebook is to train and test YOLOv4 network on BerkleyDeep Drive Dataset.

The code below was adapted from the following repositories:
1. Darknet [[Link](https://github.com/AlexeyAB/darknet)]
2. Road Object Detection using YOLOv4 [[Link](https://github.com/sourabbapusridhar/road-object-detection-using-yolov4.git)]

## Instructions to run Google Colab

1. Connect Runtime to GPU for better/faster results [RunTime -> Change RunTime Type -> GPU]
2. Download the Berkley DeepDrive dataset and the annotations after creating an account and accepting the terms to use the dataset.
3. Upload the Berkley DeepDrive dataset and the annotations to the Google Drive account connected to this Google Colab Notebook.

## 0. Prerequisites

In [ ]:
# Check whether GPU is provided
!nvidia-smi
!nvcc --version

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Check Python Version and import os to evaluate current directories
!python --version
import os

## 1. Retrain YOLOv4 on Berkley DeepDrive Dataset

### 1.1. Clone the required repositories

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content', 'Directory should be "/content" instead of "{}"'.format(os.getcwd())

# Clone the required repositories
!git clone https://github.com/AlexeyAB/darknet.git    # Official YOLOv4 Implementation
!git clone https://github.com/sourabbapusridhar/road-object-detection-using-yolov4.git      # Road Object Detection Using YoloV4 repository

### 1.2. Update Makefile based on requirements

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content', 'Directory should be "/content" instead of "{}"'.format(os.getcwd())

# Update the makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

### 1.3. Build Darknet

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Build Darknet
!make

### 1.4. Download Pre-trained weights file

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Download the pretrained weights file
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137     # For training cfg/yolov4-custom.cfg
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29     # For training cfg/yolov4-tiny-custom.cfg

# Change Directory
%cd ..

### 1.5. Setup the Berkley DeepDrive Dataset

**Important:** Please fulfill the following conditions to use the Berkley DeepDrive Dataset:
1. Register an account on the Berkley DeepDrive Website [[Link](https://bdd-data.berkeley.edu/login.html)]
2. Accept the terms to use the dataset 
3. Download the Images and the Labels and upload the zip folders into the Google Drive connected to this Colab Notebook.

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content', 'Directory should be "/content" instead of "{}"'.format(os.getcwd())

# Sanity Check: Check if the Berkley DeepDrive Dataset and the annotations exists in the Google Drive connected to this Google Colab Network
assert os.path.exists('./drive/MyDrive/bdd100k_images.zip'), 'Berkley DeepDrive Dataset does not exist! Please follow the instructions from the cell above!'
assert os.path.exists('./drive/MyDrive/bdd100k_labels_release.zip'), 'Berkley DeepDrive Annotations does not exist! Please follow the instructions from the cell above!'

# Copy the Dataset and the Annotations
!cp -rvi ./drive/MyDrive/bdd100k_images.zip ./darknet/data/
!cp -rvi ./drive/MyDrive/bdd100k_labels_release.zip ./darknet/data/

# Change Current Directory
%cd darknet

# Unzip the Dataset and the Annotations in the data folder
!unzip ./data/bdd100k_images.zip -d ./data/
!unzip ./data/bdd100k_labels_release.zip -d ./data/

# Delete unwanted files
!rm -rf ./data/bdd100k_images.zip
!rm -rf ./data/bdd100k_labels_release.zip
!rm -rf ./data/bdd100k/images/10k/

# Check Contents of the Current Directory
print("\nThe contents of the {} directory are:".format(os.getcwd()))
!ls

# Count the number of training, validation and test images
print("Number of training images: {}".format(len(os.listdir('./data/bdd100k/images/100k/train/'))))
print("Number of validation images: {}".format(len(os.listdir('./data/bdd100k/images/100k/val/'))))
print("Number of testing images: {}".format(len(os.listdir('./data/bdd100k/images/100k/test/'))))

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Copy bdd100k.names files
!cp -vi ../road-object-detection-using-yolov4/data/* ./data/bdd100k/

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Convert Labels from JSON files to text files
!python ../road-object-detection-using-yolov4/utils/convert_labels.py -ij ./data/bdd100k/labels/bdd100k_labels_images_train.json -in ./data/bdd100k/bdd100k.names -o ./data/bdd100k/images/100k/train/
!python ../road-object-detection-using-yolov4/utils/convert_labels.py -ij ./data/bdd100k/labels/bdd100k_labels_images_val.json -in ./data/bdd100k/bdd100k.names -o ./data/bdd100k/images/100k/val/

# Remove data without annotatations
!python ../road-object-detection-using-yolov4/utils/data_cleanup.py -i ./data/bdd100k/images/100k/train/
!python ../road-object-detection-using-yolov4/utils/data_cleanup.py -i ./data/bdd100k/images/100k/val/

# Generate Paths to Training and Test Images
!python ../road-object-detection-using-yolov4/utils/generate_paths.py -it data/bdd100k/images/100k/train/ -iv data/bdd100k/images/100k/val/ -o ./data/bdd100k/

# Generate data file containing relative paths to the training, validation and backup folders for YOLOv4
!python ../road-object-detection-using-yolov4/utils/generate_data_file.py -c 10 -t data/bdd100k/bdd100k_train.txt -v data/bdd100k/bdd100k_val.txt -n data/bdd100k/bdd100k.names -b backup/ -o ./data/bdd100k/

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Copy pre-defined YOLOv4 network config file to cfg folder
!cp -vi ../road-object-detection-using-yolov4/config/* ./cfg/

### 1.6. Visualize Samples from Berkley DeepDrive Dataset

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Import required libraries
import glob
import matplotlib.pyplot as plt

def read_annotations(imagePath):
  annotationsPath = imagePath.replace('.jpg', '.txt')
  with open(annotationsPath, 'rt') as annotationsFile:
    print(os.path.basename(annotationsPath) + ': \n\n' + annotationsFile.read())

imagePath = glob.glob("./data/bdd100k/images/100k/train/*.jpg")
numberOfSamples = 2       # Can be changed based on requirements

for sampleIterator in range(numberOfSamples):
  fig = plt.figure(figsize=(20,20))
  sampleImage = plt.imread(str(imagePath[sampleIterator]))
  plt.axis(False)
  plt.imshow(sampleImage)
  read_annotations(str(imagePath[sampleIterator]))
  plt.savefig("dataset_sample_{}".format(sampleIterator), dpi=300)

### 1.7. Create Symbolic Link to Google Drive

**Important:** This step is important to ensure that the weights are stored in our drive.

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Delete Backup folder
!rm -rf ./backup

# Create a New Backup Folder
!mkdir backup

# Create Symbolic links so we can save trained weights file in our Google Drive
# Create the folder YOLOv4_weight/backup in your Drive to store trained weights
!ln -svt /content/drive/MyDrive/YOLOv4_weight/ /content/darknet/backup/

### 1.8. Train YOLOv4 on Berkley DeepDrive Dataset

**Important:**
1. If you get CUDA out of memory error, adjust the number of sub-divisions in the config file
2. Adjust the number of max batches for shorter training time

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Train YOLOv4 on Custom Dataset (Berkley DeepDrive Dataset)
!./darknet detector train ./data/bdd100k/bdd100k.data ./cfg/yolov4-tiny-bdd100k.cfg ./yolov4-tiny.conv.29 -dont_show -map

### 1.9. Visualize Training Results

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())

# Plot Training Results
fig = plt.figure(figsize=(20,20))
trainingResult = plt.imread("chart.png")
plt.axis(False)
plt.imshow(trainingResult)

## Test YOLOv4 on Berkley DeepDrive Dataset

### 2.1. Test YOLOv4 on an example image

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())



### 2.2. Test YOLOv4 on an example video

In [ ]:
# Sanity Check: Check Current Directory
assert os.getcwd()=='/content/darknet', 'Directory should be "/content/darknet" instead of "{}"'.format(os.getcwd())



# TODO

In [ ]:
assert False

## Test with example image/video

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# download example images and video
!wget --no-check-certificate "https://onedrive.live.com/download?cid=A86CBC7F31A1C06B&resid=A86CBC7F31A1C06B%21120&authkey=AM5VslNNW9a8aO8" -O examples.zip
!unzip examples.zip
!rm -r examples.zip

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# cfg/coco.data was harcode in darknet, thus we need 
# to duplicate one with the same content of data/yolov4.data
%cp data/yolov4.data cfg/coco.data

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# test out our detector!
# coco.names is hardcoded somewhere in the detector
img_path = "examples/test_image_4.jpg"
!./darknet detect cfg/yolov4_custom_test.cfg backup/yolov4_custom_train_last.weights {img_path} -dont-show

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

import matplotlib.pyplot as plt 
fig = plt.figure(figsize=(12,12))
plt.axis(False)
processed_image = plt.imread("./predictions.jpg")
plt.imshow(processed_image)

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# video processed
!python darknet_video.py -v examples/test_video.mp4 -c cfg/yolov4_custom_test.cfg -w backup/yolov4_custom_train_last.weights -o output.mp4

## Processed video result

<p align="center"><img src="https://media.giphy.com/media/KyBfcsAm1VX2NVX9RV/giphy.gif" width=480></p